In [115]:
%load_ext autoreload
%autoreload 1
# %aimport transformations
# %aimport f

from pathlib import Path

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 999
pd.options.display.max_rows = 113

import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.preprocessing import Normalizer, StandardScaler, PowerTransformer

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

RS = 35577 # global random state seed
raw_data_dir_path = './data-raw'

# data_csv_path = f'{raw_data_dir_path}/bearing_signals.csv'
# lbls_csv_path = f'{raw_data_dir_path}/bearing_classes.csv'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
data_file_paths = [f for f in Path(f'{raw_data_dir_path}').iterdir()]
data_file_paths

all_df_and_name = [(pd.read_csv(csv_path), csv_path.name) for csv_path in data_file_paths]

[PosixPath('data-raw/audi.csv'),
 PosixPath('data-raw/bmw.csv'),
 PosixPath('data-raw/cclass.csv'),
 PosixPath('data-raw/focus.csv'),
 PosixPath('data-raw/ford.csv'),
 PosixPath('data-raw/hyundi.csv'),
 PosixPath('data-raw/merc.csv'),
 PosixPath('data-raw/skoda.csv'),
 PosixPath('data-raw/toyota.csv'),
 PosixPath('data-raw/unclean cclass.csv'),
 PosixPath('data-raw/unclean focus.csv'),
 PosixPath('data-raw/vauxhall.csv'),
 PosixPath('data-raw/vw.csv')]

In [84]:
for df, name in all_df_and_name:
    name, df.shape

('audi.csv', (10668, 9))

('bmw.csv', (10781, 9))

('cclass.csv', (3899, 7))

('focus.csv', (5454, 7))

('ford.csv', (17965, 9))

...

('toyota.csv', (6738, 9))

('unclean cclass.csv', (4006, 11))

('unclean focus.csv', (5604, 11))

('vauxhall.csv', (13632, 9))

('vw.csv', (15157, 9))

In [85]:
all_desc = []
for df, name in all_df_and_name:
    s = (~df.isna()).sum()
    s.name = name
    all_desc.append(s)


In [91]:
styler = pd.concat(all_desc, axis=1).style
_ = styler.background_gradient(cmap = 'Blues', axis=0, vmin=-10)
styler.format('{:.0f}')

,audi.csv,bmw.csv,cclass.csv,focus.csv,ford.csv,hyundi.csv,merc.csv,skoda.csv,toyota.csv,unclean cclass.csv,unclean focus.csv,vauxhall.csv,vw.csv
model,10668,10781,3899,5454,17965,4860,13119,6267,6738,3907,5548,13632,15157
year,10668,10781,3899,5454,17965,4860,13119,6267,6738,3904,5459,13632,15157
price,10668,10781,3899,5454,17965,4860,13119,6267,6738,3907,5548,13632,15157
transmission,10668,10781,3899,5454,17965,4860,13119,6267,6738,3907,5548,13632,15157
mileage,10668,10781,3899,5454,17965,4860,13119,6267,6738,3808,4729,13632,15157
fuelType,10668,10781,3899,5454,17965,4860,13119,6267,6738,nan,nan,13632,15157
tax,10668,10781,nan,nan,17965,nan,13119,6267,6738,nan,nan,13632,15157
mpg,10668,10781,nan,nan,17965,4860,13119,6267,6738,nan,nan,13632,15157
engineSize,10668,10781,3899,5454,17965,4860,13119,6267,6738,nan,nan,13632,15157
tax(£),nan,nan,nan,nan,nan,4860,nan,nan,nan,nan,nan,nan,nan


In [102]:
styler.data.columns

Index(['audi.csv', 'bmw.csv', 'cclass.csv', 'focus.csv', 'ford.csv',
       'hyundi.csv', 'merc.csv', 'skoda.csv', 'toyota.csv',
       'unclean cclass.csv', 'unclean focus.csv', 'vauxhall.csv', 'vw.csv'],
      dtype='object')

In [111]:
csv_names = ['audi.csv', 'bmw.csv', 'ford.csv', 'hyundi.csv', 'merc.csv', 'skoda.csv', 'toyota.csv', 'vauxhall.csv', 'vw.csv']

all_dfs_and_names = [(pd.read_csv(f), f.name[:-4]) for f in Path(f'{raw_data_dir_path}').iterdir() if f.name in csv_names]


for df, name in all_dfs_and_names:
    df.insert(0, 'make', name)

df = pd.concat(, axis=1)
    


# all_df_and_name = [(pd.read_csv(csv_path), csv_path.name) for csv_path in data_file_paths]

ValueError: All objects passed were None

In [121]:
all_df = []
for csv_name in csv_names:
    df = pd.read_csv(f'{raw_data_dir_path}/{csv_name}')
    df.insert(0, 'make', name)
    all_df.append(df)

In [126]:
df = pd.concat(all_df).drop(['tax(£)'], axis=1)

In [129]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99187 entries, 0 to 15156
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   make          99187 non-null  object 
 1   model         99187 non-null  object 
 2   year          99187 non-null  int64  
 3   price         99187 non-null  int64  
 4   transmission  99187 non-null  object 
 5   mileage       99187 non-null  int64  
 6   fuelType      99187 non-null  object 
 7   tax           94327 non-null  float64
 8   mpg           99187 non-null  float64
 9   engineSize    99187 non-null  float64
dtypes: float64(3), int64(3), object(4)
memory usage: 8.3+ MB


,year,price,mileage,tax,mpg,engineSize
count,99187.000000,99187.000000,99187.000000,94327.000000,99187.000000,99187.000000
mean,2017.087723,16805.347656,23058.914213,120.256183,55.166825,1.663280
std,2.123934,9866.773417,21148.523721,63.404805,16.138522,0.557646
min,1970.000000,450.000000,1.000000,0.000000,0.300000,0.000000
25%,2016.000000,9999.000000,7425.000000,125.000000,47.100000,1.200000
50%,2017.000000,14495.000000,17460.000000,145.000000,54.300000,1.600000
75%,2019.000000,20870.000000,32339.000000,145.000000,62.800000,2.000000
max,2060.000000,159999.000000,323000.000000,580.000000,470.800000,6.600000
